# Parallelization with Dask

In [1]:
import numpy as np
import ipyparallel as ipp
import itertools
from distributed import progress
import pandas as pd
from typing import NamedTuple

import smpsite as smp

## 1. Dask Setup

In [2]:
rc = ipp.Cluster(n=62).start_and_connect_sync()

Starting 62 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/62 [00:00<?, ?engine/s]

In [3]:
dask_client = rc.become_dask()
dask_client

<Client: 'tcp://192.168.31.89:34713' processes=0 threads=0, memory=0 B>

In [4]:
# Check to see if the threads are ready
dview = rc[:]
len(dview)

62

## 2. Macro function definition

In [5]:
def ipp_simulate_estimations(N,
                             n0,
                             kappa_within_site, 
                             site_lat,
                             site_long,
                             outlier_rate, 
                             secular_method,
                             kappa_secular, 
                             ignore_outliers, 
                             seed, 
                             n_sim):
    
    class Params(NamedTuple):
        """
        Macro to encapsulate all the parameters in the sampling model.
        """

        # Number of sites
        N : int
        # Number of samples per site
        n0 : int

        # Concentration parameter within site
        kappa_within_site : float    

        # Latitude and longitude of site
        site_lat  : float 
        site_long : float

        # Proportion of outliers to be sampled from uniform distribution
        outlier_rate : float

        # Method to sample secular variation. Options are ("tk03", "G", "Fisher")
        secular_method : str 
        kappa_secular : float    # Just needed for Fisher sampler
        
        
    params = Params(N=N,
                    n0=n0, 
                    kappa_within_site=kappa_within_site,
                    site_lat=site_lat, 
                    site_long=site_long, 
                    outlier_rate=outlier_rate, 
                    secular_method=secular_method,
                    kappa_secular=kappa_secular)
    
    
    # Create all samples
    df_tot = smp.simulate_estimations(params, 
                                      n_iters=n_sim,
                                      ignore_outliers=ignore_outliers, 
                                      seed=seed)
    
    simulation_hash = hash((N, n0, kappa_within_site, site_lat, site_long, outlier_rate, secular_method, kappa_secular))
    df_tot["hash"] = simulation_hash
    
    return df_tot

## 3. Parameter space exploration

### 3.1. Figure 1

In [17]:
min_n, max_n = 1, 300
n_simulations = 10000

params_iter = {'N': np.arange(1, 41, 1),
               'n0': np.arange(1, 41, 1), 
               'kappa_within_site': 50,
               'site_lat': [30.0],
               'site_long': [0.0], 
               'outlier_rate': [0.0],
               'secular_method': ["G"], 
               'kappa_secular': [np.nan],
               'ignore_outliers': [False]}

output_file_name = "fig1a_{}sim".format(n_simulations)

In [18]:
params_iter_mesh = np.meshgrid(*[params_iter[key] for key in params_iter.keys()])

for i, key in enumerate(params_iter.keys()):
    params_iter[key] = params_iter_mesh[i].ravel()
    
all_n_tot = params_iter['N'] * params_iter['n0']
valid_index = (min_n <= all_n_tot) & (all_n_tot <= max_n) 

n_tasks = np.sum(valid_index)
print("Total number of simulations: ", n_tasks)

indices = np.arange(n_tasks)
np.random.shuffle(indices)

for key in params_iter.keys():
    params_iter[key] = params_iter[key][valid_index]
    # Shuffle
    params_iter[key] = params_iter[key][indices]

params_iter["seed"] = np.random.randint(0, 2**32-1, n_tasks)
params_iter["n_sim"] = np.repeat(n_simulations, n_tasks)

Total number of simulations:  759


### 3.2. Figure 2

In [20]:
min_n, max_n = 50, 50

params_iter = {'N': [10, 50],
               'n0': [5, 1], 
               'kappa_within_site': np.arange(10, 101, 10),
               'site_lat': np.arange(0, 81, 10),
               'site_long': [0.0], 
               'outlier_rate': np.arange(0.10, 0.40, .02),
               'secular_method': ["G"], 
               'kappa_secular': [np.nan],
               'ignore_outliers': [False]}

In [21]:
params_iter_mesh = np.meshgrid(*[params_iter[key] for key in params_iter.keys()])

for i, key in enumerate(params_iter.keys()):
    params_iter[key] = params_iter_mesh[i].ravel()
    
all_n_tot = params_iter['N'] * params_iter['n0']
valid_index = (min_n <= all_n_tot) & (all_n_tot <= max_n)

n_tasks = np.sum(valid_index)
print("Total number of simulations: ", n_tasks)

indices = np.arange(n_tasks)
np.random.shuffle(indices)

for key in params_iter.keys():
    params_iter[key] = params_iter[key][valid_index]
    # Shuffle
    params_iter[key] = params_iter[key][indices]

params_iter["seed"] = np.random.randint(0, 2**32-1, n_tasks)

params_iter['ignore_outliers'] = params_iter['k'] == 5
params_iter["n_sim"] = np.repeat(n_simulations, n_tasks)

Total number of simulations:  2880


### Figure 2 - Function of N

In [27]:
min_n, max_n = 1, 300

params_iter = {'N': np.arange(min_n, max_n, 1),
               'n0': np.arange(1, 8, 1), 
               'kappa_within_site': 50,
               'site_lat': [30.0],
               'site_long': [0.0], 
               'outlier_rate': [0.0],
               'secular_method': ["G"], 
               'kappa_secular': [np.nan],
               'ignore_outliers': [False]}

In [28]:
params_iter_mesh = np.meshgrid(*[params_iter[key] for key in params_iter.keys()])

for i, key in enumerate(params_iter.keys()):
    params_iter[key] = params_iter_mesh[i].ravel()
    
all_n_tot = params_iter['N'] * params_iter['n0']
valid_index = (min_n <= all_n_tot) & (all_n_tot <= max_n) #& (all_n_tot % 5 == 0)

n_tasks = np.sum(valid_index)
print("Total number of simulations: ", n_tasks)

indices = np.arange(n_tasks)
np.random.shuffle(indices)

for key in params_iter.keys():
    params_iter[key] = params_iter[key][valid_index]
    # Shuffle
    params_iter[key] = params_iter[key][indices]

params_iter["seed"] = np.random.randint(0, 2**32-1, n_tasks)
params_iter["n_sim"] = np.repeat(n_simulations, n_tasks)

Total number of simulations:  776


### Figure 2 - Function of p

In [27]:
min_n, max_n = 1, 300
k_max = 5
n_simulations = 1000

params_iter = {'N': np.arange(min_n, max_n, 1),
               'n0': [1,k_max], 
               'kappa_within_site': 50,
               'site_lat': [30.0],
               'site_long': [0.0], 
               'outlier_rate': np.arange(0.0, 0.5, 0.1),
               'secular_method': ["G"], 
               'kappa_secular': [np.nan],
               'ignore_outliers': [False]}

output_file_name = "fig2b_{}sim".format(n_simulations)

In [28]:
params_iter_mesh = np.meshgrid(*[params_iter[key] for key in params_iter.keys()])

for i, key in enumerate(params_iter.keys()):
    params_iter[key] = params_iter_mesh[i].ravel()
    
all_n_tot = params_iter['N'] * params_iter['n0']
valid_index = (min_n <= all_n_tot) & (all_n_tot <= max_n) & np.logical_or(all_n_tot % k_max == 0, all_n_tot == 1)

n_tasks = np.sum(valid_index)
print("Total number of simulations: ", n_tasks)

indices = np.arange(n_tasks)
np.random.shuffle(indices)

for key in params_iter.keys():
    params_iter[key] = params_iter[key][valid_index]
    # Shuffle
    params_iter[key] = params_iter[key][indices]

params_iter["ignore_outliers"] = params_iter["n0"] == k_max
params_iter["seed"] = np.random.randint(0, 2**32-1, n_tasks)
params_iter["n_sim"] = np.repeat(n_simulations, n_tasks)

Total number of simulations:  600


### Figure 3 - Intersection

In [6]:
min_n, max_n = 40, 40

params_iter = {'N': [8, 40],
               'n0': [1, 5], 
               'kappa_within_site': 50,
               'site_lat': [30.0],
               'site_long': [0.0], 
               'outlier_rate': np.arange(0, 0.61, 0.02),
               'secular_method': ["G"], 
               'kappa_secular': [np.nan],
               'ignore_outliers': [False]}

In [7]:
params_iter_mesh = np.meshgrid(*[params_iter[key] for key in params_iter.keys()])

for i, key in enumerate(params_iter.keys()):
    params_iter[key] = params_iter_mesh[i].ravel()
    
all_n_tot = params_iter['N'] * params_iter['n0']
valid_index = (min_n <= all_n_tot) & (all_n_tot <= max_n) #& np.logical_or(all_n_tot % 10 == 0, all_n_tot == 1)

n_tasks = np.sum(valid_index)
print("Total number of simulations: ", n_tasks)

indices = np.arange(n_tasks)
np.random.shuffle(indices)

for key in params_iter.keys():
    params_iter[key] = params_iter[key][valid_index]
    # Shuffle
    params_iter[key] = params_iter[key][indices]

params_iter["ignore_outliers"] = params_iter["n0"] == 5
params_iter["seed"] = np.random.randint(0, 2**32-1, n_tasks)
params_iter["n_sim"] = np.repeat(n_simulations, n_tasks)

Total number of simulations:  62


### Figure 3 - Heatmap

## 4. Run Simulation

In [29]:
task = dask_client.map(ipp_simulate_estimations, 
                       params_iter['N'],
                       params_iter['n0'],
                       params_iter['kappa_within_site'], 
                       params_iter['site_lat'], 
                       params_iter['site_long'], 
                       params_iter['outlier_rate'],
                       params_iter['secular_method'], 
                       params_iter['kappa_secular'], 
                       params_iter['ignore_outliers'], 
                       params_iter['seed'], 
                       params_iter["n_sim"])

res = dask_client.submit(pd.concat, task)

progress(res)

VBox()

In [30]:
%%time
df_all = res.result()

ValueError: No points to compute the mean

In [31]:
df_all

,plong,plat,total_samples,samples_per_sites,S2_vgp,error_angle,S2_vgp_real,n_tot,N,n0,kappa_within_site,site_lat,site_long,outlier_rate,secular_method,kappa_secular,ignore_outliers,hash
0,69.937306,87.686607,25.0,25,NaN,2.313393,191.7229,25,1,25,50,30.0,0.0,0.0,G,NaN,False,-5088040779576061458
1,301.477273,79.432550,25.0,25,NaN,10.567450,191.7229,25,1,25,50,30.0,0.0,0.0,G,NaN,False,-5088040779576061458
2,15.847469,60.814243,25.0,25,NaN,29.185757,191.7229,25,1,25,50,30.0,0.0,0.0,G,NaN,False,-5088040779576061458
3,350.751130,75.870259,25.0,25,NaN,14.129741,191.7229,25,1,25,50,30.0,0.0,0.0,G,NaN,False,-5088040779576061458
4,235.044441,87.936368,25.0,25,NaN,2.063632,191.7229,25,1,25,50,30.0,0.0,0.0,G,NaN,False,-5088040779576061458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,313.097714,82.600680,8.0,4,27.826346,7.399320,191.7229,8,2,4,50,30.0,0.0,0.0,G,NaN,False,7045401061585812115
9996,135.435703,86.012562,8.0,4,113.537378,3.987438,191.7229,8,2,4,50,30.0,0.0,0.0,G,NaN,False,7045401061585812115
9997,93.791165,75.949697,8.0,4,402.762860,14.050303,191.7229,8,2,4,50,30.0,0.0,0.0,G,NaN,False,7045401061585812115
9998,216.028959,75.158947,8.0,4,8.774578,14.841053,191.7229,8,2,4,50,30.0,0.0,0.0,G,NaN,False,7045401061585812115


In [32]:
df_all.to_csv('outputs/' + output_file_name + '_total.csv')

In [33]:
# Summary table
df_summary = df_all.groupby("hash", as_index=False).apply(smp.summary_simulations)
df_summary

,,error_angle_mean,error_angle_median,error_angle_25,error_angle_75,error_angle_95,error_angle_std,error_angle_S2,error_vgp_scatter,n_tot,N,n0,kappa_within_site,site_lat,site_long,outlier_rate,secular_method,kappa_secular,ignore_outliers,total_simulations
0,0,4.147206,3.912595,2.484667,5.486852,8.148171,2.186134,21.978023,3.186314,26,13,2,50,30.0,0.0,0.0,G,NaN,False,10000
1,0,4.725067,4.445762,2.876439,6.279026,9.143663,2.455932,28.357256,2.932066,189,7,27,50,30.0,0.0,0.0,G,NaN,False,10000
2,0,3.980893,3.739991,2.413162,5.281213,7.779722,2.085578,20.196707,2.438957,190,10,19,50,30.0,0.0,0.0,G,NaN,False,10000
3,0,3.233462,3.013762,1.950203,4.281603,6.409835,1.715021,13.396281,5.554613,29,29,1,50,30.0,0.0,0.0,G,NaN,False,10000
4,0,2.158836,2.034649,1.311973,2.846312,4.181181,1.125675,5.927591,1.309937,296,37,8,50,30.0,0.0,0.0,G,NaN,False,10000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,0,2.649010,2.473473,1.605169,3.503009,5.164014,1.387456,8.942095,1.581143,264,24,11,50,30.0,0.0,0.0,G,NaN,False,10000
755,0,2.606820,2.440106,1.561114,3.462770,5.143039,1.368585,8.668347,1.647996,135,27,5,50,30.0,0.0,0.0,G,NaN,False,10000
756,0,2.316476,2.175341,1.420605,3.062309,4.489364,1.200979,6.808266,1.450202,170,34,5,50,30.0,0.0,0.0,G,NaN,False,10000
757,0,6.289641,5.932100,3.841007,8.335166,12.206067,3.266839,50.230759,4.174381,92,4,23,50,30.0,0.0,0.0,G,NaN,False,10000


In [34]:
df_summary.to_csv('outputs/' + output_file_name + '_summary.csv')